In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd
import torch
import copy
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch import Tensor

In [2]:
train = pd.read_csv('dataAll.csv')
test = pd.read_csv('testAll.csv')
train_data=train.iloc[:,:30].applymap(lambda x: x+126.23)
train_loc = train.iloc[:,30:]

In [3]:
train_feat = torch.from_numpy(
        train_data.values.astype(np.float32))
train_labels = torch.from_numpy(
        train_loc.values.astype(np.float32))

In [4]:
num_train = int(0.8 * train_feat.shape[0]) # 划分训练样本和验证集样本
indices = np.arange( train_feat.shape[0])
np.random.shuffle(indices)  # shuffle 顺序
train_indices = indices[:num_train]
valid_indices = indices[num_train:]

train_features = train_feat[train_indices,:]
train_label = train_labels[train_indices,:]
valid_features  = train_feat[valid_indices,:]
valid_labels = train_labels[valid_indices,:]


In [5]:
feat_dim = train_data.shape[0]

In [6]:
def error(y_test,y_pre):
    y_test = np.array(y_test)
    y_pre = np.array(y_pre)
    m = y_test-y_pre
    m=np.square(m)
    m=m.sum(axis=1)
    m=np.sqrt(m)
    return m.sum()/len(y_test)

def get_model(feat_dim,output_dim = 2 ):
    net = nn.Sequential(
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim)),
        nn.BatchNorm1d(int(feat_dim)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim), int(feat_dim*4/5)),
        nn.BatchNorm1d(int(feat_dim*4/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*4/5),int(feat_dim*3/5)),
        nn.BatchNorm1d(int(feat_dim*3/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*3/5),int(feat_dim*2/5)),
        nn.BatchNorm1d(int(feat_dim*2/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim*2/5),int(feat_dim/5)),
        nn.BatchNorm1d(int(feat_dim/5)),
        nn.ReLU(inplace=True),
        
        nn.Linear(int(feat_dim/5), output_dim)
        )
    return net

def get_data(x,y,batch_size,shuffle):
    dataset = TensorDataset(x,y)
    return DataLoader(dataset, batch_size, shuffle=shuffle,num_workers=4)

def train_model(model,x_train,y_train,x_valid,y_valid,epochs,batch_size,lr,weight_decay,use_gpu):
    
    if use_gpu:
        model = model.cuda()
    train_data = get_data(x_train,y_train,batch_size,True)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.MSELoss()
    #criterion = nn.CrossEntropyLoss()
    best_error = 0
    for e in  range(epochs):
        model.train()
        for data in train_data:
            x,y = data
            if use_gpu:
                x = x.cuda()
                y = y.cuda()
                
            out = model(x)
            loss = criterion(out, y)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if use_gpu:
                x_valid = x_valid.cuda()
        with torch.no_grad():
            output = model(x_valid)
        output = Tensor.cpu(output)
        out = output.numpy()
        print('epoch:{}'.format(e))
        error_i = error(out,y_valid)
        print('error:{}'.format(error_i)) 
        if e == 0:
            best_error = error_i
        elif error_i < best_error:
            best_model = copy.deepcopy(model)
    torch.save(best_model,"model.t7")

def predict(x_test):
    net = torch.load('./model.t7')
    a = net(x_test).detach().numpy()
    a = np.argmax(a,axis=1)
    with open('./ans.txt','w') as f:
        for i in a:
            f.write('{}'.format(i))
            f.write('\n')
    

In [7]:
# 可以调整的超参
batch_size = 128
epochs = 10000
lr = 0.001
wd = 0
use_gpu = False
feat_dim = train_feat.shape[1]
net = get_model(feat_dim)
train_model(net, train_features, train_label, valid_features, valid_labels, epochs, 
            batch_size, lr, wd, use_gpu)
#predict(x_test)

epoch:0
error:37.21633449223803
epoch:1
error:37.125886865297545
epoch:2
error:37.03016352684347
epoch:3
error:36.92523548673997
epoch:4
error:36.81953428201811
epoch:5
error:36.690289456662356
epoch:6
error:36.54630396992238
epoch:7
error:36.39007923673997
epoch:8
error:36.21764836675291
epoch:9
error:36.034989489003884
epoch:10
error:35.83319352360932
epoch:11
error:35.614643131468306
epoch:12
error:35.373041821636484
epoch:13
error:35.14524882761966
epoch:14
error:34.89709330530401
epoch:15
error:34.55371220084088
epoch:16
error:34.18636299320828
epoch:17
error:33.802949142949544
epoch:18
error:33.41299118693402
epoch:19
error:33.001473055465716
epoch:20
error:32.57449415831177
epoch:21
error:32.116715010510994
epoch:22
error:31.653738478331178
epoch:23
error:31.202038526843467
epoch:24
error:30.674659403298836
epoch:25
error:30.266976774741266
epoch:26
error:29.75008590717982
epoch:27
error:29.30077619663648
epoch:28
error:28.78328903217982
epoch:29
error:28.27322525873221
epoch:30

epoch:241
error:2.230623825092982
epoch:242
error:2.1347264614327295
epoch:243
error:1.9916193124393597
epoch:244
error:1.9938933260228007
epoch:245
error:1.9843907917609962
epoch:246
error:1.9192047711331663
epoch:247
error:1.8996777849086353
epoch:248
error:1.9443097231767466
epoch:249
error:1.8622124636157826
epoch:250
error:1.8529561229079075
epoch:251
error:1.843660302797542
epoch:252
error:1.897188687540427
epoch:253
error:1.9348394041073738
epoch:254
error:1.8417842415912031
epoch:255
error:1.8099303646507114
epoch:256
error:1.8753628946676908
epoch:257
error:1.8752390872614813
epoch:258
error:1.8291959972307568
epoch:259
error:1.8133767585705045
epoch:260
error:1.84395371371685
epoch:261
error:1.909778195241753
epoch:262
error:1.922318748483991
epoch:263
error:1.872448683093467
epoch:264
error:1.8453677079964423
epoch:265
error:1.8059866881771507
epoch:266
error:1.8850618784362871
epoch:267
error:1.9470960214869018
epoch:268
error:2.040125917185479
epoch:269
error:1.89290580404

epoch:478
error:1.6281191886319535
epoch:479
error:1.5738719629285252
epoch:480
error:1.6405015084290104
epoch:481
error:1.5943017641292043
epoch:482
error:1.6821020602563066
epoch:483
error:1.7095509833845408
epoch:484
error:1.6176781500141495
epoch:485
error:1.5844789729543984
epoch:486
error:1.5956655406088291
epoch:487
error:1.62239625444696
epoch:488
error:1.6049989451103654
epoch:489
error:1.688708385551423
epoch:490
error:1.6515816396143272
epoch:491
error:1.524784221377749
epoch:492
error:1.5364521219073415
epoch:493
error:1.58609999722065
epoch:494
error:1.751306610304819
epoch:495
error:1.607303752627749
epoch:496
error:1.621443695423674
epoch:497
error:1.621643619117885
epoch:498
error:1.603691229382277
epoch:499
error:1.5380088737063389
epoch:500
error:1.5850902720225581
epoch:501
error:1.5910099715495634
epoch:502
error:1.6075039921571799
epoch:503
error:1.6468761685903137
epoch:504
error:1.6305602790467335
epoch:505
error:1.5419005952862226
epoch:506
error:1.5694546915426

epoch:714
error:1.4994785560519082
epoch:715
error:1.45926894253315
epoch:716
error:1.5220762502021346
epoch:717
error:1.5305561100218306
epoch:718
error:1.5399042008610933
epoch:719
error:1.5801412920439846
epoch:720
error:1.6564091809508408
epoch:721
error:1.5556242672622898
epoch:722
error:1.5126739936226552
epoch:723
error:1.5323101009156694
epoch:724
error:1.5375790219720247
epoch:725
error:1.4830042751455368
epoch:726
error:1.5105362629366106
epoch:727
error:1.52791320090152
epoch:728
error:1.5106820208906049
epoch:729
error:1.4923310471074547
epoch:730
error:1.547274215717982
epoch:731
error:1.6197926668115297
epoch:732
error:1.522847361891575
epoch:733
error:1.5303827164860933
epoch:734
error:1.512309203943645
epoch:735
error:1.538240696757762
epoch:736
error:1.4885405507155562
epoch:737
error:1.5122262971984153
epoch:738
error:1.4354803916862064
epoch:739
error:1.54081175336554
epoch:740
error:1.5046291968184022
epoch:741
error:1.4765883984880337
epoch:742
error:1.581743524114

epoch:950
error:1.4974458405764877
epoch:951
error:1.4711896691562905
epoch:952
error:1.4781097767423999
epoch:953
error:1.4900439263623868
epoch:954
error:1.513948072899822
epoch:955
error:1.5105503176038972
epoch:956
error:1.5343785057709411
epoch:957
error:1.5228686807689198
epoch:958
error:1.484244244218952
epoch:959
error:1.5528307047420764
epoch:960
error:1.4829704807770052
epoch:961
error:1.495011698536546
epoch:962
error:1.5492646092739328
epoch:963
error:1.4830418595367076
epoch:964
error:1.4605196500040427
epoch:965
error:1.4479509874272316
epoch:966
error:1.5129407164658797
epoch:967
error:1.5491111133570505
epoch:968
error:1.4745070443887451
epoch:969
error:1.431066120734961
epoch:970
error:1.4731470579317594
epoch:971
error:1.5550102835947606
epoch:972
error:1.5240022133732212
epoch:973
error:1.5273728068402328
epoch:974
error:1.4782687997756307
epoch:975
error:1.5513170644505982
epoch:976
error:1.5171973856423835
epoch:977
error:1.6119986431718951
epoch:978
error:1.526333

...

epoch:8992
error:1.316601278248302
epoch:8993
error:1.2002527155512208
epoch:8994
error:1.16064484708522
epoch:8995
error:1.153409693867238
epoch:8996
error:1.199020010687864
epoch:8997
error:1.4084497923067594
epoch:8998
error:1.2566214853856728
epoch:8999
error:1.614741514139311
epoch:9000
error:1.4423685999151035
epoch:9001
error:1.3704387077538809
epoch:9002
error:1.3302425172319696
epoch:9003
error:1.326850762805223
epoch:9004
error:1.2528453594962807
epoch:9005
error:1.2328687028773853
epoch:9006
error:1.2207703189430386
epoch:9007
error:1.2238369999696799
epoch:9008
error:1.2142039468032422
epoch:9009
error:1.1949412356989813
epoch:9010
error:1.1967790808437095
epoch:9011
error:1.1865984483647316
epoch:9012
error:1.187325579999798
epoch:9013
error:1.172144012648569
epoch:9014
error:1.2199110892272396
epoch:9015
error:1.1903189872554172
epoch:9016
error:1.168745783599814
epoch:9017
error:1.1695707451942512
epoch:9018
error:1.1586328188167043
epoch:9019
error:1.1669331263138745
ep

epoch:9222
error:1.112166035838454
epoch:9223
error:1.1192884358829236
epoch:9224
error:1.1594374090394566
epoch:9225
error:1.190140935150186
epoch:9226
error:1.1404891908554333
epoch:9227
error:1.1394795446161465
epoch:9228
error:1.133492177393273
epoch:9229
error:1.1442683961382196
epoch:9230
error:1.131058667023771
epoch:9231
error:1.133626328402935
epoch:9232
error:1.143436407210139
epoch:9233
error:1.134387886076973
epoch:9234
error:1.1494776243380094
epoch:9235
error:1.1584123658331986
epoch:9236
error:1.1690736205580934
epoch:9237
error:1.165514947217618
epoch:9238
error:1.1453787938126616
epoch:9239
error:1.1560404433315814
epoch:9240
error:1.1302119127991592
epoch:9241
error:1.160884881852361
epoch:9242
error:1.1904630870745068
epoch:9243
error:1.1710672514250484
epoch:9244
error:1.2313734599874677
epoch:9245
error:1.1801405656229786
epoch:9246
error:1.1934947103917368
epoch:9247
error:1.1658430210523125
epoch:9248
error:1.1691665550715555
epoch:9249
error:1.1444877436984557
e

epoch:9452
error:1.1801076398013017
epoch:9453
error:1.1532432487063389
epoch:9454
error:1.159590194327094
epoch:9455
error:1.1495178143697444
epoch:9456
error:1.1263636975208198
epoch:9457
error:1.2054741034069776
epoch:9458
error:1.151090042094518
epoch:9459
error:1.151301730650671
epoch:9460
error:1.1567930786606566
epoch:9461
error:1.1400074631862467
epoch:9462
error:1.1117992721893193
epoch:9463
error:1.1547581523386967
epoch:9464
error:1.3609551236052717
epoch:9465
error:1.1963315423370795
epoch:9466
error:1.1353136780653703
epoch:9467
error:1.1676198310407908
epoch:9468
error:1.1363924922178201
epoch:9469
error:1.1308044196717335
epoch:9470
error:1.1953008140968628
epoch:9471
error:1.2409146840637129
epoch:9472
error:1.1724582686923917
epoch:9473
error:1.230089510859678
epoch:9474
error:1.1997809366914618
epoch:9475
error:1.1880849268272962
epoch:9476
error:1.1523262211452943
epoch:9477
error:1.1796000926028865
epoch:9478
error:1.1852693348004932
epoch:9479
error:1.1315745838555

epoch:9682
error:1.1509987657159606
epoch:9683
error:1.1438958684963212
epoch:9684
error:1.163586615282382
epoch:9685
error:1.153202663880579
epoch:9686
error:1.166805055132196
epoch:9687
error:1.1573522649175292
epoch:9688
error:1.151396086422623
epoch:9689
error:1.1302940299563389
epoch:9690
error:1.1594674133853493
epoch:9691
error:1.234591031290427
epoch:9692
error:1.210680331172178
epoch:9693
error:1.219038673391009
epoch:9694
error:1.177803621871968
epoch:9695
error:1.168570416093952
epoch:9696
error:1.1438405183740297
epoch:9697
error:1.1638898960523125
epoch:9698
error:1.1586291867116754
epoch:9699
error:1.168269267211756
epoch:9700
error:1.179655442725178
epoch:9701
error:1.1375105488963455
epoch:9702
error:1.1693005481636076
epoch:9703
error:1.311393629224612
epoch:9704
error:1.2031583995745068
epoch:9705
error:1.1757452447849288
epoch:9706
error:1.1462348651762613
epoch:9707
error:1.1351703678343306
epoch:9708
error:1.124141638831056
epoch:9709
error:1.1336853106302556
epoch

epoch:9912
error:1.144453396618289
epoch:9913
error:1.1371644724541154
epoch:9914
error:1.164196729968467
epoch:9915
error:1.161885763664295
epoch:9916
error:1.1799775946494988
epoch:9917
error:1.1898131771507114
epoch:9918
error:1.19809982477462
epoch:9919
error:1.1602315766999516
epoch:9920
error:1.1927202034736821
epoch:9921
error:1.1759685402854139
epoch:9922
error:1.1686436898649741
epoch:9923
error:1.1241706167124839
epoch:9924
error:1.133833595266009
epoch:9925
error:1.1384999816815573
epoch:9926
error:1.1470481408675615
epoch:9927
error:1.1707909745664213
epoch:9928
error:1.130248470725865
epoch:9929
error:1.143237431021588
epoch:9930
error:1.1237278946929576
epoch:9931
error:1.1222931342476552
epoch:9932
error:1.1761474609375
epoch:9933
error:1.133137889235325
epoch:9934
error:1.1121220557840799
epoch:9935
error:1.1497250022740135
epoch:9936
error:1.133960245189198
epoch:9937
error:1.1659275859324467
epoch:9938
error:1.1424004676887936
epoch:9939
error:1.1067281429394404
epoch